In [1]:
import matplotlib.pyplot as plt
from kronbinations import *
from unipolator import *
import numpy as np
from numpy import empty, arange, zeros, mean, std, linspace, random, diag, array, ones, logspace
from tqdm import tqdm
import pickle

from Analysis_Code.parameter_sweep import *
from Analysis_Code.useful import *
from Analysis_Code.iplots import *
import Analysis_Code.discrete_quantum as dq
from Analysis_Code.Average_Infidelity_Integrator import *

In [5]:
# Test autobinning methods
n = 2
s = 16
amps = np.pi/2 * array([1.0,0.025])
I_tar = 1e-10

rng = np.random.default_rng(100)
H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
c_maxs = ones(n)
c_mins = -c_maxs
#ui = UI(H_s, c_mins, c_maxs, np.ones(n, dtype=int))
#ui_auto = UI_auto(H_s, c_mins, c_maxs, I_tar)
UI_bins(H_s, c_mins, c_maxs, I_tar)

array([12, 10])

In [ ]:
# 1D interpolation vs Trotter and Strang, with distribution
n = 1 # 1D interpolation
s = 16 # * 4
amps = np.pi/2 * array([1.0, 0.025]) 
redo = False

bins = array([1,2,4,8])
mini = 10**-15
maxi = 1.0
do_figures = 1
rng = random.default_rng(123)
markersize = 4
width = 8.5
reps = 100
plot_log = 1 # 1=semilogy; 2=loglog
ps = 0.68

how_many = 8*15+1
c_s = linspace(mini,maxi,how_many)
n_bins = len(bins)

c_mins = ones(n)*mini
c_maxs = ones(n)*maxi

I = empty([reps, n_bins, how_many])
I_trotter = empty([reps, how_many])
I_strang = empty([reps, how_many])

U_ui = empty([s,s], dtype=complex)
U_classic = empty([s,s], dtype=complex)
U_trotter = empty([s,s], dtype=complex)
U_strang = empty([s,s], dtype=complex)

filename = 'one_d_interpolation_gauss.pkl'
if filename in os.listdir('Cache') and not redo:
    # Load the data
    print('Loading data from file')
    with open(os.path.join('Cache', filename), 'rb') as f:
        I_mean, I_std_min, I_std_max, I_trotter_mean, I_trotter_std_min, I_trotter_std_max, I_strang_mean, I_strang_std_min, I_strang_std_max = pickle.load(f)
else:
    print('Computing data')
    for rep in tqdm(range(reps)):
        #H_s = dq.Random_parametric_Hamiltonian_Haar(n, s, amps, rng=rng)
        H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
        ui = []
        for i, bin in enumerate(bins):
            c_bins = bin*np.ones(n, dtype=int)
            # Prepare interpolations and exact decompositions
            ui.append(UI(H_s, c_mins, c_maxs, c_bins))
        system = Hamiltonian_System(H_s)
        trotter = Trotter_System(H_s)
        strang = Sym_Trotter_System(H_s)
        for i, d in enumerate(c_s):
            c = np.array([d]) #np.random.random(n)
            system.expmH(c, U_classic)
            # 1 Bin
            for j in range(len(bins)):
                ui[j].expmH(c, U_ui)
                I[rep,j,i] = dq.Av_Infidelity(U_ui, U_classic)
            c = np.array([d]) #np.random.random(n)
            system.expmH(c, U_classic)
            trotter.expmH(c, U_trotter)
            strang.expmH(c, U_strang)
            I_trotter[rep,i] = dq.Av_Infidelity(U_trotter, U_classic)
            I_strang[rep,i] = dq.Av_Infidelity(U_strang, U_classic)
            
    # Extract mean and std from the repetitions
    I_mean, I_std_min, I_std_max = mean_std_asym(I, axis=0)
    I_trotter_mean, I_trotter_std_min, I_trotter_std_max = mean_std_asym(I_trotter, axis=0)
    I_strang_mean, I_strang_std_min, I_strang_std_max = mean_std_asym(I_strang, axis=0)
    # Save the data
    with open(os.path.join('Cache', filename), 'wb') as f:
        pickle.dump([I_mean, I_std_min, I_std_max, I_trotter_mean, I_trotter_std_min, I_trotter_std_max, I_strang_mean, I_strang_std_min, I_strang_std_max], f)

# Plot the results
figsize = np.array([10,10])  # width, height
fig = plt.figure(figsize=figsize)
ax1 = fig.add_subplot(111)

legendary = []
for j, bin in enumerate(bins):
    legendary, _ = min_mean_max_plot(ax1, [c_s], [I_mean[j,:]], [I_mean[j,:]-I_std_min[j,:]], [I_mean[j,:]+I_std_max[j,:]], ps=ps, label=r'UI('+str(bin)+' bins)', legendary=legendary, colors='C'+str(j), support=[10**-16,1.0])

ax1.set(xlim=[mini, maxi], ylabel=r'$I=1-F$', xlabel=r'$c$')
legendary, _ = min_mean_max_plot(ax1, c_s, [I_trotter_mean], [I_trotter_mean-I_trotter_std_min], [I_trotter_mean+I_trotter_std_max], ps=ps, label=r'Trotter', legendary=legendary, colors='C'+str(n_bins), support=[10**-16,1.0])
legendary, _ = min_mean_max_plot(ax1, c_s, [I_strang_mean], [I_strang_mean-I_strang_std_min], [I_strang_mean+I_strang_std_max], ps=ps, label=r'Sym Trotter', legendary=legendary, colors='C'+str(n_bins+1), support=[10**-16,1.0])
ax1.legend(*legendary, loc='upper right')
y_lim = ax1.get_ylim()
if plot_log >= 1:
    ax1.set(ylim=[10**-13, y_lim[1]], yscale='log')
    logstr = '_log'
else:
    ax1.set(ylim=[0, y_lim[1]])
    logstr = ''
if y_lim[1]*1.15 > 1-1/(s+1):
    ax1.hlines(1-1/(s+1), mini, maxi, linestyles='dotted', colors='black', zorder=-1)
for i in range(1,max(bins)):
    ax1.axvline(i/max(bins), color='gray', linestyle='dotted')
plt.show()
filename = 'one_d_'+str(bins[0])+'_bins_infidelity_comparison_std_plot'+logstr
fig2tikz(fig, filename, legend_columns=3, add_letter='a')

In [ ]:
# 1D derivatives (matrix)
n = 1 # 1D interpolation
s = 16
eps = 10**(-8)
bins = array([1,2,4,8])
mini = 0.0
maxi = 1.0
do_figures = 1
rng = random.default_rng(123)
markersize = 4
width = 8.5
reps = 100
plot_log = 1 # 1=semilogy; 2=loglog
ps = 0.68
redo = False

c_mins = ones(n)*mini
c_maxs = ones(n)*maxi

U_ui = empty([s,s], dtype=complex)
dU_ui = empty([n,s,s], dtype=complex)
U_classic = empty([s,s], dtype=complex)
U_trotter = empty([s,s], dtype=complex)
U_trotter2 = empty([s,s], dtype=complex)
dU_classic = empty([n,s,s], dtype=complex)
how_many = 16*7+1
c_s = linspace(mini,maxi,how_many)
n_bins = len(bins)

I_ui = empty([n_bins, how_many, reps])
I_trotter = empty([how_many, reps])
I_strang = empty([how_many, reps])

filename = 'one_d_interpolation_gauss_derivative.pkl'
if filename in os.listdir('Cache') and not redo:
    # Load the data
    print('Loading data from file')
    with open(os.path.join('Cache', filename), 'rb') as f:
        I_ui_mean, I_ui_std_min, I_ui_std_max, I_trotter_mean, I_trotter_std_min, I_trotter_std_max, I_strang_mean, I_strang_std_min, I_strang_std_max = pickle.load(f)
else:
    print('Computing data')
    for r in tqdm(range(reps)):
        amps = np.pi/2 * array([1.0, 0.025])
        H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
        v0 = dq.rand_state(s, rng=rng)
        dv_ui = empty([1,s,1], dtype=complex)

        ui = []
        for i, bin in enumerate(bins):
            c_bins = bin*np.ones(n, dtype=int)
            # Prepare interpolations and exact decompositions
            ui.append(UI(H_s, c_mins, c_maxs, c_bins))
        system = Hamiltonian_System(H_s)
        trotter = Trotter_System(H_s)
        strang = Sym_Trotter_System(H_s)
        for i, d in enumerate(c_s):
            c = np.array([d]) #np.random.random(n)
            c2 = np.array([d+eps]) #np.random.random(n)
            system.dexpmH(c, U_classic, dU_classic)
            trotter.expmH(c, U_trotter)
            trotter.expmH(c2, U_trotter2)
            dU_trotter = (U_trotter2-U_trotter)/eps
            strang.expmH(c, U_trotter)
            strang.expmH(c2, U_trotter2)
            dU_strang = (U_trotter2-U_trotter)/eps
            I_trotter[i,r] = np.mean(np.abs(dU_classic[0] - dU_trotter))**2   
            I_strang[i,r] = np.mean(np.abs(dU_classic[0] - dU_strang))**2   
            for j in range(len(bins)):
                ui[j].dexpmH(c, U_ui, dU_ui)
                I_ui[j,i,r] = np.mean(np.abs(dU_classic[0] - dU_ui[0,]))**2   

    I_ui_mean, I_ui_std_min, I_ui_std_max = mean_std_asym(I_ui, axis=2)
    I_trotter_mean, I_trotter_std_min, I_trotter_std_max = mean_std_asym(I_trotter, axis=1)
    I_strang_mean, I_strang_std_min, I_strang_std_max = mean_std_asym(I_strang, axis=1)
    # Save data
    with open(os.path.join('Cache', filename), 'wb') as f:
        pickle.dump([I_ui_mean, I_ui_std_min, I_ui_std_max, I_trotter_mean, I_trotter_std_min, I_trotter_std_max, I_strang_mean, I_strang_std_min, I_strang_std_max], f)
        
fig, ax1 = plt.subplots(figsize=(10, 10))
legendary = []
for i in range(len(bins)):
    # use min_mean_ax_plot to plot the mean and std
    mea, mini, maxi = I_ui_mean[i,:], I_ui_std_min[i,:], I_ui_std_max[i,:]
    legendary, _ = min_mean_max_plot(ax1, c_s, [mea], [mea-mini], [mea+maxi], ps=ps, label=r'UI ('+str(bins[i])+' bins)', legendary=legendary, support=[10**-15,1.0])

mea, mini, maxi = I_trotter_mean, I_trotter_std_min, I_trotter_std_max
legendary, _ = min_mean_max_plot(ax1, c_s, [mea], [mea-mini], [mea+maxi], ps=ps, label=r'Trotter', legendary=legendary, support=[10**-15,1.0])
mea, mini, maxi = I_strang_mean, I_strang_std_min, I_strang_std_max
legendary, _ = min_mean_max_plot(ax1, c_s, [mea], [mea-mini], [mea+maxi], ps=ps, label=r'Sym Trotter', legendary=legendary, support=[10**-15,1.0])

ax1.legend(*legendary)
ax1.set_xlabel(r'$c$')
ax1.set_ylabel(r'$\vert \partial_c U_{\mathrm{exact}} - \partial_c U_{\mathrm{approx}} \vert ^2 $')
#$\left\vert \frac{\partial U_{\mathrm{exact}}}{\partial c} - \frac{\partial U_{\mathrm{approx}}}{\partial c} \right\vert ^2 $')
ax1.set_xlim([0,1])
ax1.set_ylim([10**(-13),10**-3])
ax1.set_yscale('log')

for i in range(1,max(bins)):
    ax1.axvline(i/max(bins), color='gray', linestyle='dotted')
    
plt.show()
filename = 'one_d_'+str(bins[0])+'_bins_partial_infidelity_comparison_std'
fig2tikz(fig, filename, legend_columns=3, add_letter='b')

In [ ]:
# Combine into a single plot!
file_list = ['one_d_1_bins_infidelity_comparison_std_plot_log', 'one_d_1_bins_partial_infidelity_comparison_std']
output_filename = 'one_d_figures_combined'
combine_tex_files(file_list, output_filename, columns=2, directory='Tikz', redo=False, columns_half=1.2)

In [ ]:
n = 2 # 2D interpolation
bins = 5
s = 16
do_figures = 1 # required for this one
figsize = np.array([20,10])  # width, height
ratios = np.array([1, 1])
how_many = 1000
redo = False

def infidelities_2d(k, bins, n, s, I, I_sym, rng):
    amps = np.pi/2*np.array([1.0, 0.025])
    #H_s = Random_parametric_Hamiltonian_Haar(n, s, amps, rng=rng)
    H_s = Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
    
    c_mins = np.ones(n) * 0
    c_maxs = np.ones(n) * 1.0
    c_bins = bins*np.ones(n, dtype=int)
    ui = UI(H_s, c_mins, c_maxs, c_bins)
    sym_ui = Sym_UI(H_s, c_mins, c_maxs, c_bins)
    system = Hamiltonian_System(H_s)

    U = np.empty([s,s], dtype=complex)
    U_sym = np.empty([s,s], dtype=complex)
    U_classic = np.empty([s,s], dtype=complex)
    for i, (d,e) in k.kronprod(change=False, progress=True):
        c = np.array([d, e]) #np.random.random(n-1)
        ui.expmH(c, U)
        sym_ui.expmH(c, U_sym)
        system.expmH(c, U_classic)
        I[i] = Av_Infidelity(U, U_classic)
        I_sym[i] = Av_Infidelity(U_sym, U_classic)
    return I, I_sym
c_s = np.linspace(0.0, 1.0 ,how_many)
k = JIT_kronbinations(c_s, c_s, func=infidelities_2d, redo= redo, import_statements=['from Analysis_Code.discrete_quantum import *', 'unipolator'], other_arguments=[bins, n, s])
I = k.empty()
I_sym = k.empty()
k.calculate_all()

fig1 = plt.figure(figsize=figsize)
ax1 = fig1.add_subplot(111)
fig2 = plt.figure(figsize=figsize)
ax2 = fig2.add_subplot(111)
axs = [ax1, ax2]
figs = [fig1, fig2]
names = ['UI', 'Sym UI']
Is = [I.array(), I_sym.array()]
max_I = max([I.max(), I_sym.max()])
a = ['a', 'b']
for j, (fig, ax, cI, name)  in enumerate(zip(figs, axs, Is, names)): 
    p = ax.imshow(cI, extent=(0.0, 1.0, 0.0, 1.0), interpolation='spline16', origin='lower') #, norm=LogNorm(vmin=10**-15, vmax=max_I))
    #p.set_clim(10**-15, max_I)
    fig.colorbar(p, ax=ax, label='$I=1-F$', fraction=0.046, pad=0.04)#, orientation='horizontal')
    # Horizontal and Vertical plots
    for i in range(bins+1):
        c = 1/bins*i
        ax.plot([0.0, 1.0], [c, c], color='C0', linestyle='dotted')
        ax.plot([c, c], [0.0, 1.0], color='C0', linestyle='dotted')
    # Diagonal plots
    for i in range(0,2*bins+1, 2):
        c =  1/bins*i
        c2 =  1/bins*(i - bins - 1)
        ax.plot([c, 0], [0, c], color='C1', linestyle='dotted')
        ax.plot([c2, c2 + 1.0], [0.0, 1.0], color='C1', linestyle='dotted')
    ax.set(xlabel=r'$c_1$', ylabel=r'$c_2$', xlim=[0.0, 1.0], ylim=[0.0, 1.0])
    plt.set_cmap('viridis')
    # axis ratio equal
    ax.set_aspect('equal', 'box')
    ax.set_title(name)
    if do_figures:
        filename = 'two_d_'+str(bins)+'_bins_infidelity_comparison_'+ name.replace(' ', '_')
        fig2tikz(fig, filename, axis_width=7, axis_height=7, add_letter=a[j])

# Print results mean
line()
print('Mean UI: ', num2expstr(I.mean()))
print('Mean Sym UI: ', num2expstr(I_sym.mean()))
# Ratio
print('Ratio: ', num2expstr(I.mean()/I_sym.mean()))


In [ ]:
# Combine into a single plot!
file_list = ['two_d_5_bins_infidelity_comparison_UI', 'two_d_5_bins_infidelity_comparison_Sym_UI']
output_filename = 'two_d_infidelities_combined'
combine_tex_files(file_list, output_filename, columns=2, directory='Tikz', redo=False, shift_by=2.5)

In [ ]:
def plot_infidelities(method_dicts, axis_labels, axis_types, axis_vars, plot_which=['mean'], figsize=(8,8), ps=0.68, mini = 10**-15, ylims = None, ylog=True, indexes=None, vline=None):
    legendary = []
    #generate figure with one axis
    fig = plt.figure(figsize=figsize)
    ax1 = fig.add_subplot(111)
    if not vline is None:
        if ylims is None: # avoid this as color argument "gray" is not preserved by tikzplotlib
            ax1.axvline(vline, color='gray', linestyle='dashed')
        else:
            ax1.plot([vline, vline], ylims, color='gray', linestyle='dashed')
    if indexes is None:
        indexes = np.arange(len(axis_vars[0]))
    x_vars = axis_vars[0][indexes]
    for method, method_dict in method_dicts.items():
        if not method == 'times':
            for plot_str in plot_which:
                # get the correct elements out
                mean_vals = method_dict[plot_str+'_I_mean'][indexes,0]
                std_vals_min = method_dict[plot_str+'_I_std_min'][indexes,0]
                std_vals_max = method_dict[plot_str+'_I_std_max'][indexes,0]
                # plot the mean and std
                if 'log' in axis_types[0]:
                    legendary, _ = min_mean_max_plot(ax1, x_vars, [mean_vals], [mean_vals-std_vals_min], [mean_vals+std_vals_max], ps=ps, label=method, legendary=legendary, support=[10**-16,1.0])
                else: # errorbarplot with caps of size 5 and linestyle '--'
                    ax1.errorbar(x_vars, mean_vals, yerr=(std_vals_min, std_vals_max), fmt='o', capsize=5, linestyle='--', label=method)
    # set the axis labels
    ax1.set_xlabel(axis_labels[0])
    ax1.set_ylabel('$I = 1 - F$')
    ax1.set(xlim=(x_vars[0], x_vars[-1]))
    if ylog:
        ax1.set_yscale('log')
    if 'log' in axis_types[0]:
        ax1.set_xscale('log')
    # get ylim
    if ylims is None:
        ylim = ax1.get_ylim()
        if ylim[0] < mini:
            ylim = (mini, ylim[1])
        if ylim[1] > 1:
            ylim = (ylim[0], 1)
    else:
        ylim = ylims.copy()
    ax1.set(ylim=ylim)
    # if not log scale, set the ticks to integer values of axis_vars
    if 'log' not in axis_types[0]:
        ax1.set_xticks(axis_vars[0])
    # set the legend
    ax1.legend(*legendary)
    # Add axis ticks on the right and top  -> also ticks inside 
    ax1.tick_params(direction='in', top=True, right=True)
    # show plot
    return fig, ax1

In [ ]:
# specify the variables by a dictionary 
num_controls = {'label': 'Number of controls', 'name': 'num_controls', 'def_val': 2, 'variation_vals': np.arange(1, 8), 'variation_type': 'int'}
hilbert_dim = {'label':'Hilbert space dimension', 'name': 'dim_hilbert', 'def_val': 16, 'variation_vals': 2**np.arange(1, 10), 'variation_type': 'log_int', 'weight': lambda x: (2**x)**3 }
control_amplitudes = {'label': 'Eigenvalue ratio', 'name': 'amp_ratio', 'def_val': 0.025, 'variation_vals': np.logspace(-3, 0, 61), 'variation_type': 'log'} # ([H_i>0| / |H_0|)
bins = {'label': 'Number of bins', 'name': 'bins', 'def_val': 1, 'variation_vals': np.arange(1,64), 'variation_type': 'log_int'}
all_parameter_dicts = [num_controls, hilbert_dim, control_amplitudes, bins] 

import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], ['numpy', 'np'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange, initialize_rep_arrays, intitialize_integrators, mean_std, mean_std_asym]

In [ ]:

# Methods to use 
m_times_list    = [0]#,1,2,3,4,5,6,7,8,9,10]
m_times_list_ui = [0]#,1,2,3,4,5,6,7,8,9,10] 
method_dict = {}
method_dict['UI'] = {'method': UI, 'm_times': m_times_list_ui}
method_dict['Sym UI'] = {'method': Sym_UI, 'm_times': m_times_list_ui}
method_dict['Trotter'] = {'method': Trotter_System, 'm_times': m_times_list}
method_dict['Sym Trotter'] = {'method': Sym_Trotter_System, 'm_times': m_times_list}

redo=False
combinations = ['num_controls', 'amp_ratio']#, 'bins']  
to_do = ['max', 'mean']
letters = ['b', 'a']
ylims_list = [[10**-9, 10**-2], [10**-15, 10**-0]]
for i, (combination, letter, ylims) in enumerate(zip(combinations, letters, ylims_list)):
    parameters, axis_labels, axis_types = get_parameters(combination, all_parameter_dicts)  
    k = JIT_kronbinations(parameters, func=Infidelities_and_Times, other_func=other_func, import_statements=import_statements, other_arguments=[method_dict, to_do], redo=redo)
    method_dict_results, method_dict = create_variables(k, method_dict, to_do=to_do)
    k.calculate_all()
    line('Done with '+combination)
    indexes = np.where(parameters['dim_hilbert'] > 2)[0] if 'dim_hilbert' == combination else None
    # Plot the results
    axis_vars = [item for key, item in parameters.items() if key in combination]
    for plot_which in ['mean']:
        filename = plot_which+'_infidelity_by_'+combination 
        v_line = None
        vline = 2 if 'num_controls' == combination else 0.025 if 'amp_ratio' == combination else None
        fig, ax1 = plot_infidelities(method_dict_results, axis_labels, axis_types, axis_vars, plot_which=[plot_which], figsize=(8,8), ylog=True, ylims=ylims, indexes=indexes, vline=vline)
        plt.show()
        fig2tikz(fig, filename, add_letter=letter)

In [ ]:
num_controls = {'label': 'Number of controls', 'name': 'num_controls', 'def_val': 2, 'variation_vals': np.arange(1, 3), 'variation_type': 'int'}
hilbert_dim = {'label':'Hilbert space dimension', 'name': 'dim_hilbert', 'def_val': 16, 'variation_vals': 2**np.arange(1, 10), 'variation_type': 'log_int'}
control_amplitudes = {'label': 'Eigenvalue ratio', 'name': 'amp_ratio', 'def_val': 0.025, 'variation_vals': np.logspace(-3, -2, 61), 'variation_type': 'log'} # ([H_i>0| / |H_0|)
bins = {'label': 'Number of bins', 'name': 'bins', 'def_val': 1, 'variation_vals': np.arange(1,64), 'variation_type': 'log_int'}
all_parameter_dicts = [num_controls, hilbert_dim, control_amplitudes, bins] 
import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], ['numpy', 'np'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange, initialize_rep_arrays, intitialize_integrators, mean_std]
m_times_list_ui = [0,1,2,3,4,5,6,7,8,9,10] 
m_times_list    = [0,1,2,3,4,5,6,7,8,9,10]
method_dict = {}
method_dict['UI'] = {'method': UI, 'm_times': m_times_list_ui}
method_dict['Sym UI'] = {'method': Sym_UI, 'm_times': m_times_list_ui}
method_dict['Trotter'] = {'method': Trotter_System, 'm_times': m_times_list}
method_dict['Sym Trotter'] = {'method': Sym_Trotter_System, 'm_times': m_times_list}
redo = False
to_do = ['mean', 'max']   
combinations = ['num_controls']
for i, combination in enumerate(combinations):
    parameters, axis_labels, axis_types = get_parameters(combination, all_parameter_dicts)  
    k = JIT_kronbinations(parameters, func=Infidelities_and_Times, other_func=other_func, import_statements=import_statements, other_arguments=[method_dict, to_do], redo=redo, progress=True)
    method_dict_results, method_dict = create_variables(k, method_dict, to_do=to_do)
    k.calculate_all()

In [ ]:
from scipy.optimize import curve_fit
# plot the infidelity as a function of the number of trotter steps and the number of bins to reach a target fidelity  --> load the apropatirate data ('amp_ratio')
# fig, ax single axis
# import curve_fit
save = True
# at which amp_ratio to plot ? --> allow change via slider
for controls in [1,2]:
    control_index, controls = closest_match(parameters['num_controls'], controls)

    # for every method, plot the infidelity as a function of the number of trotter steps (or the number of bins if UI)
    ps = 0.68
    for plot_which in ['mean']:
        legendary = []
        fig, ax = plt.subplots(1,1, figsize=(8,8))
        for name, data in method_dict_results.items():
            if not name == 'times':
                if 'UI' in name: # use bins
                    x_vals = 2**np.array(m_times_list_ui)
                else:
                    x_vals = 2**np.array(m_times_list)
                # plot mean values using std_plot
                # extract values
                mean_I, std_I_min, std_I_max = data[plot_which + '_I_mean'][control_index,:], data[plot_which + '_I_std_min'][control_index,:], data[plot_which + '_I_std_max'][control_index,:]
                if 'Sym Trotter' in name:
                    # Fit a curve to mean_I[-3:] 
                    def model(x, c):
                        return c*x
                    # fit the model , set initital values
                    popt, pcov = curve_fit(model, x_vals[-3:], mean_I[-3:], p0=[1.0])
                    # plot the fit
                    ax.plot(x_vals, model(x_vals, *popt), linestyle='--', color='gray')
                legendary, _ = min_mean_max_plot(ax, x_vals, [mean_I], [mean_I - std_I_min], [mean_I + std_I_max], ps=ps, label=name, legendary=legendary, support=[10**-15,1.0], )
        # xlabel
        ax.set_xlabel(r'Number of Trotter Steps (Bins)')
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_ylabel(r'$I_{\mathrm{'+plot_which+'}}$')
        ax.legend(*legendary, loc='best')
        ylim = ax.get_ylim()
        ax.set_ylim([10**-15, ylim[1]])
        ax.set_xlim([1, 2**max(m_times_list)])
        # set xticks to 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024
        x_vals_ticks = 2**np.array(m_times_list)[::2]
        ax.set_xticks(x_vals_ticks)
        ax.set_xticklabels([str(x) for x in x_vals_ticks])
        #ax.set(title=r'Control amplitude ratio ($|H_i| / |H_0|$) of '+str(round_sig(amp_ratio,1)))
        filename = plot_which + '_Steps2infidelity_with_'+str(controls)+'_controls'
        plt.show()
        if save:
            fig2tikz(fig, filename, add_letter='c')
        line()

In [ ]:
checksum_pre = 'pgi_'
## Calculate the time it takes to get to a target fidelity via kronbinations parameter sweep
redo = False
# Variation by different variables, 
# specify the variables by a dictionary 
num_controls = {'label': 'Number of controls', 'name': 'num_controls', 'def_val': 2, 'variation_vals': np.arange(1, 3), 'variation_type': 'int'}
hilbert_dim = {'label':'Hilbert space dimension', 'name': 'dim_hilbert', 'def_val': 16, 'variation_vals': 2**np.arange(1, 10), 'variation_type': 'log_int'}
control_amplitudes = {'label': 'Eigenvalue ratio', 'name': 'amp_ratio', 'def_val': 0.025, 'variation_vals': np.logspace(-3, 0, 61), 'variation_type': 'log'} # ([H_i>0| / |H_0|)
bins = {'label': 'Number of bins', 'name': 'bins', 'def_val': 1, 'variation_vals': np.arange(1,64), 'variation_type': 'log_int'}
all_parameter_dicts = [num_controls, hilbert_dim, control_amplitudes, bins] 
# Methods to use 
m_times_list_ui = [0,1,2,3,4,5,6,7] 
m_times_list    = [0,1,2,3,4,5,6,7,8,9,10]
method_dict = {}
method_dict['UI'] = {'method': UI, 'm_times': m_times_list_ui}
method_dict['Sym UI'] = {'method': Sym_UI, 'm_times': m_times_list_ui}
method_dict['Trotter'] = {'method': Trotter_System, 'm_times': m_times_list}
method_dict['Sym Trotter'] = {'method': Sym_Trotter_System, 'm_times': m_times_list}

import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange, initialize_rep_arrays, intitialize_integrators]
combinations = ['num_controls']  
to_do = ['max', 'mean', 'times'] 
for i, combination in enumerate(combinations):
    parameters, axis_labels, axis_types = get_parameters(combination, all_parameter_dicts)  
    k = JIT_kronbinations(parameters, func=Binned_Infidelities_and_Times, other_func=other_func, import_statements=import_statements, other_arguments=[method_dict, to_do], redo=redo, checksum_pre=checksum_pre)
    method_dict_results, method_dict = create_variables(k, method_dict, to_do=to_do)
    print(k.checksum)
    k.calculate_all()
    line('Done with '+combination)

In [ ]:
import scipy
# plot the infidelity as a function of the number of trotter steps and the number of bins to reach a target fidelity  --> load the apropatirate data ('amp_ratio')
# at which amp_ratio to plot ? --> allow change via slider
min_x = 10**-16
for curr_num_controls in [1, 2]:
    num_controls_index, num_controls = closest_match(parameters['num_controls'], curr_num_controls)
    # for every method, plot the infidelity as a function of the number of trotter steps (or the number of bins if UI)
    ps = 0.68
    legendary = []
    min_x = 10**-15
    lineparameters = {} # fit a line to the data (in logarithm, so t(I) = a + b*exp(I)
    f = lambda x, a, b: a + b*np.log(x)
    df = lambda x, a, b, da, db: np.sqrt(da**2 + (np.log(x)*db)**2)
    for plot_which in ['mean']: #, 'max']:
        line(checksum_pre[:-1] + ' ' + plot_which)
        fig, ax = plt.subplots(1,1, figsize=(8,8))
        # Determine the maximum value of the infidelity
        for d, (name, data) in enumerate(method_dict_results.items()):
            if not name == 'times':
                # extract values
                t_vals = data['times'][num_controls_index,:]
                # Calculate the partial minima 
                mean_I, std_I_min, std_I_max = data[plot_which+'_I_mean'][num_controls_index,:], data[plot_which+'_I_std_min'][num_controls_index,:], data[plot_which+'_I_std_max'][num_controls_index,:]
                # Calculate the partial minima 
                # Double the va
                mean_indexes = index_partial_minimum(mean_I)#, std_I_min)
                mean_Is = mean_I[mean_indexes]
                min_Is =  std_I_min[mean_indexes] 
                max_Is =  std_I_max[mean_indexes]
                ts = t_vals[mean_indexes]
                # without line connecting draw errorbars
                for i in range(len(mean_Is)):
                    if mean_Is[i]- min_Is[i] < min_x:
                        min_Is[i] = mean_Is[i] - min_x
                ax.errorbar(mean_Is, ts, xerr=(min_Is, max_Is), capsize=5, capthick=2, label=name, fmt='o', markersize=2, markeredgewidth=2, color='C'+str(d))
                # fit f to the data using scipy.optimize.curve_fit
                fit_param, covariance = scipy.optimize.curve_fit(f, mean_Is, ts, p0=[0.1,0.1])
                std_param = np.sqrt(np.diag(covariance))
                # save in lineparameters
                lineparameters[name] = {'param': fit_param, 'std_param': std_param}
        for d, (name, data) in enumerate(method_dict_results.items()):
            if not name == 'times':
                I = data[plot_which+'_I_mean'][num_controls_index,:]
                min_I, max_I = np.min(I), np.max(I)
                a, b = lineparameters[name]['param']
                da, db = lineparameters[name]['std_param']
                x = np.array([min_x, max_I])
                ax.semilogx(x, f(x, a,b), linestyle=':', color='C'+str(d))
                # evaluate error at 10**-15 (min_x) using error propagation
                t_at_min_I = f(min_I, a,b)
                dt_at_min_I = df(min_I, a,b, da, db)
                print(name+': a = ' + plusminus(a, da,1) + ', b = ' + plusminus(b, db,1) + ', t(I=10**-15) = ' + plusminus(t_at_min_I, dt_at_min_I,1)) 
        ax.set_ylabel(r't [s]')
        ax.set_xscale('log')
        ax.set_xlabel(r'$I=1-F$')
        ax.legend(*legendary, loc='best')
        xlim = ax.get_xlim()
        ax.set_xlim([min_x, xlim[1]])
        # show figure
        plt.show()
        filename = checksum_pre+plot_which+'_time2infidelity_'+str(num_controls)
        fig2tikz(fig, filename, add_letter='d')


In [ ]:
file_list = ['mean_infidelity_by_amp_ratio',
'mean_infidelity_by_num_controls', 
'mean_Steps2infidelity_with_2_controls',
'pgi_mean_time2infidelity_2']

output_filename = 'infidelities_and_speed_combined'
combine_tex_files(file_list, output_filename, columns=2, directory='Tikz', redo=True, shift_by_y=0.5, columns_half=1.18)

In [ ]:
## Load the times it takes as a function of the Hilbert space dimension
# Variation by different variables, 
# specify the variables by a dictionary 
num_controls = {'label': 'Number of controls', 'name': 'num_controls', 'def_val': 2, 'variation_vals': np.arange(1, 3), 'variation_type': 'int'}
hilbert_dim = {'label':'Hilbert space dimension', 'name': 'dim_hilbert', 'def_val': 16, 'variation_vals': 2**np.arange(1, 11), 'variation_type': 'log_int', 'weight': lambda x: (2**x)**3}
control_amplitudes = {'label': 'Eigenvalue ratio', 'name': 'amp_ratio', 'def_val': 0.025, 'variation_vals': np.logspace(-3, -1, 23), 'variation_type': 'log'} # ([H_i>0| / |H_0|)
bins = {'label': 'Number of bins', 'name': 'bins', 'def_val': 1, 'variation_vals': np.arange(1,64), 'variation_type': 'log_int'}
all_parameter_dicts = [num_controls, hilbert_dim, control_amplitudes, bins] 
import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], ['numpy', 'np'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange, initialize_rep_arrays, intitialize_integrators, mean_std]
m_times_list_ui = [0]
m_times_list    = [0]
method_dict = {}
method_dict['UI'] = {'method': UI, 'm_times': m_times_list_ui}
method_dict['Sym UI'] = {'method': Sym_UI, 'm_times': m_times_list_ui}
method_dict['Trotter'] = {'method': Trotter_System, 'm_times': m_times_list}
method_dict['Sym Trotter'] = {'method': Sym_Trotter_System, 'm_times': m_times_list}

import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], ['numpy', 'np'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange, initialize_rep_arrays, intitialize_integrators, mean_std]
combination = 'dim_hilbert num_controls'
to_do = ['times'] 
parameters, axis_labels, axis_types = get_parameters(combination, all_parameter_dicts)  

devices = ['pgi_'] 
arrays = []
names = ['UI', 'Sym UI', 'Trotter', 'Sym Trotter', 'Eigh']
keys = ['UI_times', 'Sym UI_times', 'Trotter_times', 'Sym Trotter_times', 'times']
for checksum_pre in devices:
    k = JIT_kronbinations_load(parameters, func=Binned_Infidelities_and_Times, import_statements=import_statements, other_arguments=[method_dict, to_do], checksum_pre=checksum_pre)
    arrays.append(k.get_arrays())


In [ ]:
# plot the time to execute a step as a function of the dimension of the Hilbert space
letters = ['a', 'b']
for i, (device, array) in enumerate(zip(devices, arrays)):
    for curr_num_controls, curr_letter in zip([1,2], letters):
        num_controls_index, num_controls = closest_match(parameters['num_controls'], curr_num_controls)
        dims = parameters['dim_hilbert']
        # for every method, plot the infidelity as a function of the number of trotter steps (or the number of bins if UI)
        fig, ax = plt.subplots(1,1, figsize=(10,10))
        # Determine the maximum value of the infidelity
        for d, (key, name) in enumerate(zip(keys, names)):
            t_vals = array[key][num_controls_index,:,0] 
            ax.loglog(dims, t_vals, label=name)
        ax.set_ylabel(r't [s]')
        #ax.set_xscale('log')
        ax.set_xlabel(r'Dimension of Hilbert space')
        xlim = ax.get_xlim()
        ax.set_xlim([min(dims), max(dims)])
        # Title is replaced by comment in the caption
        #ax.set(title=curr_num_controls) #r'Control amplitude ratio ($|H_i| / |H_0|$) of '+str(round_sig(amp_ratio,2)))
        # Add legend
        ax.legend()
        # show figure
        plt.show()
        
        filename = checksum_pre+'times_by_dim_hilbert_num_controls_'+str(curr_num_controls)
        fig2tikz(fig, filename, add_letter=curr_letter)


In [ ]:
file_list = ['pgi_times_by_dim_hilbert_num_controls_1', 'pgi_times_by_dim_hilbert_num_controls_2']

output_filename = 'times_by_dim_combined'
combine_tex_files(file_list, output_filename, columns=2, directory='Tikz', redo=False, columns_half=1.185)